In [1]:
from pickle import load
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer, f1_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from sklearn import svm

In [2]:
df = load(open('df_clean_tweet_tokens.pkl', 'rb'))

In [3]:
df.head()

,text,tokenized,company,target,product
0,wesley83 i have a 3g iphone after 3 hrs tweeti...,. @wesley83 I have a 3G iPhone . After 3 hrs t...,apple,0,iphone
1,jessedee know about fludapp awesome ipadiphon...,@jessedee Know about @fludapp ? Awesome iPad /...,apple,2,ipad or iphone app
2,swonderlin can not wait for ipad 2 also they s...,@swonderlin Can not wait for #iPad 2 also . Th...,apple,2,ipad
3,sxsw i hope this years festival isnt as crashy...,@sxsw I hope this year's festival isn't as cra...,apple,0,ipad or iphone app
4,sxtxstate great stuff on fri sxsw marissa maye...,@sxtxstate great stuff on Fri #SXSW : Marissa ...,google,2,google


In [4]:
X = df['text']
y = df.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [6]:
sw = stopwords.words('english')

In [7]:
# dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)

In [8]:
micro_recall = make_scorer(recall_score, average = 'micro')

In [9]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'macro_precision' : make_scorer(precision_score, average = 'macro'),
           'macro_recall' : make_scorer(recall_score, average = 'macro'), 
           'macro_f1_score' : make_scorer(f1_score, average = 'macro')}
# pipe = make_pipeline(TfidfVectorizer(max_features=300, stop_words= sw), DecicionTreeClassifier())
# cv = cross_validate(pipe, X_train, y_train, scoring = scoring)
# cv

In [10]:
# y_hat_train = cross_val_predict(pipe, X_train, y_train)

In [11]:
# confusion_matrix(y_train, y_hat_train)

In [12]:
# , 'decisiontreeclassifier__max_depth': np.arange(3, 15)

In [13]:
param = {'svc__kernel': ['rbf', 'poly', 'linear']}
pipe2 = make_pipeline(TfidfVectorizer(stop_words= sw), svm.SVC())
grid = GridSearchCV(pipe2, param, scoring= micro_recall)

In [14]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfidfvectorizer',
                                        TfidfVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
          

In [15]:
grid.best_score_

0.676253081347576

In [16]:
grid.best_params_

{'svc__kernel': 'linear'}

In [17]:
grid.best_estimator_

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('svc', SVC(kernel='linear'))])

In [18]:
y_pred_acc = grid.predict(X_test)

In [19]:
confusion_matrix(y_test,y_pred_acc)

array([[ 25,  90,  26],
       [  6, 935, 207],
       [  2, 331, 407]], dtype=int64)